In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
import numpy as np 
import torch

In [ ]:
def plot_y(y_pred, y_true):
    y_pred = y_pred.reshape(-1)
    y_true = y_true.reshape(-1)
    order = np.argsort(y_pred)
    plt.plot(y_true[order], 'o ', label=u'réel')
    plt.plot(y_pred[order], 'P ', label=u'prédiction')
    plt.legend()
    plt.xlabel(u'échantillons')
    plt.ylabel('valeur de y')

# Optimization d'un MLP

Dans cet exercice nous allons réaliser l'optimization d'un perceptron à une couche caché. 
On se servira de pytorch et sa fonctionalité autograd pour apliquer la descente du gradient.


## Les données : Predire la progression du diabetes

On continuera à travailler avec les données sur la progression du diabetes. Le code ci dessus charge les données dans des variables, les normalize et les divide en "train" et "test", exactement comme dans le notebook antérieur.

In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes()

#print(data.DESCR)

X, y = data.data, data.target

# import pandas as pd
# pd.DataFrame(X[:10,:], columns=data.feature_names)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

y_train, y_test = y_train.reshape([-1,1]), y_test.reshape([-1,1])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## MLP en pytorch

Ici je reprends le code pour le MLP qui à été fait dans le notebook antérieur.

In [ ]:
M = X_train.shape[0]  # nombre d'échantillons
D = X_train.shape[1]  # nombre de features
S = y_train.shape[1]  # nombre de sorties

In [ ]:
def parameters(D, S, N=5):
    rng = np.random.RandomState(0)
    
    W = torch.from_numpy(rng.rand(D,N))
    b = torch.zeros((N,1), dtype=torch.float64)
    
    O = torch.from_numpy(rng.rand(N,S))

    return W, b, O

In [ ]:
def MLP(X, W, b, O):
    M, D = X.shape  # nombre d'échantillons, nombre de features
    N = W.shape[1]  # N le nombre de neurones de la couche caché
    
    # On transforme X en un torch.Tensor
    X = torch.from_numpy(X)
    
    # np.dot devient torch.mm (matrix multiplication)
    H = torch.mm(X,W)+b.transpose(1,0)
    A = torch.tanh(H)
    Y = torch.mm(A,O)

    
    # Ici quelques verifications sur la taille des matrices pour vous aider
    try:
        assert(H.shape == (M,N))
    except AssertionError:
        print("Taille de H semble erronée:",H.shape, ", ça devrait être", (M,N))    

    try:
        assert(Y.shape == (M,S))
    except AssertionError:
        print("Taille de Y semble erronée:",Y.shape, ", ça devrait être", (M,S))
    
    return Y

In [ ]:
def cout(Y_pred, Y_true):
    M = Y_true.shape[0]
    Y_true = torch.from_numpy(Y_true)
    # completez le code avec l'expression pour la fonction de cout J
    J = (1/M)*((Y_pred-Y_true)**2).sum()
    return J

In [ ]:
W, b, O = parameters(D, S, N=5)

In [ ]:
Y_pred = MLP(X_train, W, b, O)

In [ ]:
J = cout(Y_pred, y_train)
J

## Retro-propagation avec Autograd

Ici on va implementer la retro-propagation en s'apuiant sur autograd. Pour cela il faut habiliter le calcul de gradient sur les tensors qui determinent nos paramètres W, b et O.
Pour cela, il faudra maquer leur attribut `requires_grad` comme vrai: `requires_grad=True`.
Ceci est fait ci-dessus pour chacun des paramètres.

In [ ]:
def parameters(D, S, N=5):
    rng = np.random.RandomState(0)
    
    # ici on met requires_grad = True sur le tensor directement
    W = torch.from_numpy(rng.rand(D,N))
    W.requires_grad = True

    # ici on peu le passer en tant que mot-cle dans la création de b
    b = torch.zeros((N,1), dtype=torch.float64, requires_grad=True)
    
    # une 3eme façon de le faire ici
    O = torch.from_numpy(rng.rand(N,S))
    O.requires_grad_(True)

    return W, b, O

On peut les re-créer avec ce nouveau format:

In [ ]:
W, b, O = parameters(D, S, N=5)

Et ensuite recalculer les predicions et la fonction de cout:

In [ ]:
Y_pred = MLP(X_train, W, b, O)

In [ ]:
J = cout(Y_pred, y_train)
J

### La fonction `backward`
En apelant la methode backward sur la fonction de cout, les gradients seront calculés:

In [ ]:
J.backward(retain_graph=True)

Maintenant on peut consulter les valeurs obtenues pour les gradients
- `W.grad` est $\nabla_W J$
- `O.grad`est  $\nabla_O J$ 
- et `b.grad` est $\nabla_b J$:

In [ ]:
W.grad

In [ ]:
b.grad

In [ ]:
O.grad

La fonction `backward` en fait calcule les gradients et les somme aux valeurs qui sont déjà dedans les variables `grad` de chaque paramètre. Pour cela, si on veut les recalculer, il faut d'abord remettre les variables `grad` à zéro, comme fait dans la fonction ci-dessous:

In [ ]:
def zero_grads(W, b, O):
    W.grad.data.zero_()
    b.grad.data.zero_()
    O.grad.data.zero_()

Si on appèlle cette fonction, les gradients seront mis à zéro, comme vous pouvez le vérifier ici.

In [ ]:
zero_grads(W, b, O)

In [ ]:
W.grad, b.grad, O.grad

### Exercice : pas du gradient

Maintenant vous savez comment calculer les gradients et voir leurs valeurs. A vous de remplir la fonction cidessus avec les mises à jour pour la descente du gradient.
Vous allez metre à jour W, b et O.

In [ ]:
def gradient_step(learning_rate, W,b,O):
    lr = learning_rate
    # Metez à jour W, b et O 
    # dans la direction oposé du gradient
    # Ca prendra la forme
    # Tensor.data = Tensor.data - lr * Tensor.grad
    W.data = W.data - lr * W.grad



### Exercice : boucle d'optimization
Tout est prêt pour l'entrainement de notre réseau. Il faut maintenant créer la boucle d'optimization qui realise la descente de gradient pour W, b, et O. Suivez les indications et completez le code ci-dessous.

In [ ]:
W, b, o = parameters(D, S, N=5) # N est le nombre de neurones de la couche caché
max_iterations = 1000
learning_rate = 1e-5
cost_curve = []
for i in range(max_iterations):
    #Completez le code ci dessous (numeros 1 a 5)
    # 1) calculez les predictions avec le réseau (forward pass)
    Y_pred
    
    # 2) Calculez la fonction de cout
    J = 
    
    # Sauvegarder J pour plot
    cost_curve.append(J.detach().numpy())
    
    # 3) Calculez les gradients avec la fonction backward()
    

    # 4) metez a jour W, b et o avec la fonction gradient_step
    
    
    # 5) mettez à zero les variables grad avant le prochain pas
    #    avec la fonction zero_grads()
    
    

Si tout se passe bien vous allez voir ci-dessus l'evolution de la valeur de la focntion de coût au long des iterations.

In [ ]:
plt.plot(cost_curve)
plt.title("courbe d'apprentissage")
plt.xlabel('iterations')

In [ ]:
plot_y(Y_pred.detach().numpy(), y_train)

## Évaluation du modèle

Finalement, on poura tester la qualité de notre modéle sur notre ensemble de teste ( qui n'a pas été utilisé pour l'apprentissage).

In [ ]:
Y_pred = MLP(X_test, W, b, O)
J = cout(Y_pred,y_test)
print("cout sur l'ensemble de test", J.detach().numpy())
plot_y(Y_pred.detach().numpy(), y_test)

### Exercice : essayez d'augmenter le nombre d'iterations, de neurones ou changer la learning rate pour voir si le modèle peut mieux faire !

## Corrigés
### Gradient step
``` python
def gradient_step(learning_rate, W,b,O):
    lr = learning_rate
    # Metez à jour W, b et O 
    # dans la direction oposé du gradient
    # Ca prendra la forme
    # Tensor.data = Tensor.data - lr * Tensor.grad
    W.data = W.data - lr * W.grad
    b.data = b.data - lr * b.grad
    O.data = O.data - lr * O.grad
```    
### Boucle d'entrainement
``` python
for i in range(max_iterations):
    #Completez le code ci dessous (numeros 1 a 5)
    # 1) calculez les predictions avec le réseau (forward pass)
    Y_pred = MLP(X_train, W, b, O)
    
    # 2) Calculez la fonction de cout
    J = (Y_pred, y_train)
    
    # Sauvegarder J pour plot
    cost_curve.append(J.detach().numpy())
    
    # 3) Calculez les gradients avec la fonction backward()
    J.backward()

    # 4) metez a jour W, b et o avec la fonction gradient_step
    gradient_step(W, b, o)
    
    # 5) mettez à zero les variables grad avant le prochain pas
    #    avec la fonction zero_grads()
    zero_grads(W, b, o)
```     
    